In [27]:
import ast
import pandas as pd
from datasets import load_dataset

# Loading Data
dataset = load_dataset('lukebarousse/data_jobs')
df = dataset['train'].to_pandas()

# Cleaning Data
df['job_posted_date'] = pd.to_datetime(df['job_posted_date'])
df['job_skills'] = df['job_skills'].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)

# Filtering Data for US
df_US = df[df['job_country'] == 'United States']

In [28]:
df_US_skills = df_US.explode('job_skills')

In [29]:
df_skills_count = df_US_skills.groupby(['job_title_short', 'job_skills']).size()
df_skills_count = df_skills_count.reset_index(name='skill_count')
df_skills_count.sort_values(by='skill_count', ascending=False, inplace=True)

In [30]:
job_titles = df_skills_count['job_title_short'].unique().tolist()
job_titles = sorted(job_titles[:3])

In [31]:
df_job_title_count = df_US['job_title_short'].value_counts().reset_index(name='job_count')

In [33]:
df_skills_pct = pd.merge(df_skills_count, df_job_title_count, on='job_title_short', how='left')

In [36]:
df_skills_pct['skill_pct'] = df_skills_pct['skill_count'] / df_skills_pct['job_count'] * 100